# Gitops example

# Install deployment controller with gitops enabled

In [8]:
! helm install mlflow-controller-staging ../../charts/mlflow-controller  -n mlflow --set image.tag=f20fd19f28f1f39ced794e0a2f7736f403447d91 --set gitops.enabled=true   --set mlflow.backend=blob --set gitops.repository=github.com/rocket9-code/model-deployments   --set gitops.deploymentLocation=staging --set mlflow.stage=Staging \--set mlflow.namespace=staging

NAME: mlflow-controller-staging
LAST DEPLOYED: Mon Dec 19 14:29:32 2022
NAMESPACE: mlflow
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [11]:
! kubectl wait --for=condition=ready pod -l 'app.kubernetes.io/instance in (mlflow-controller-staging)' --timeout=180s -n mlflow

pod/mlflow-controller-staging-787fd66687-gxl8z condition met


In [ ]:
! kubectl port-forward -n mlflow svc/mlflow-service 5000:5000 

# Register Mlflow models

In [12]:
import os

import mlflow
import mlflow.sklearn
import pandas as pd
from minio import Minio
from mlflow.tracking import MlflowClient
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"
os.environ["AZURE_STORAGE_ACCESS_KEY"] = ""
os.environ["AZURE_STORAGE_CONNECTION_STRING"] = ""


def main(MODEL_NAME="iris gitops", stage="Staging"):

    iris = datasets.load_iris()
    iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
    y = iris.target
    iris_df["target"] = y

    print(iris_df.head())

    train_df, test_df = train_test_split(
        iris_df, test_size=0.3, random_state=42, stratify=iris_df["target"]
    )
    X_train = train_df[
        [
            "sepal length (cm)",
            "sepal width (cm)",
            "petal length (cm)",
            "petal width (cm)",
        ]
    ]
    y_train = train_df["target"]

    X_test = test_df[
        [
            "sepal length (cm)",
            "sepal width (cm)",
            "petal length (cm)",
            "petal width (cm)",
        ]
    ]
    y_test = test_df["target"]

    EXPERIMENT_NAME = MODEL_NAME

    print("IRIS train df shape")
    print(X_train.shape)
    print(y_train.shape)

    print("IRIS test df shape")
    print(X_test.shape)
    print(y_test.shape)

    mlflow_client = MlflowClient()

    # Create an MLFlow experiment, if not already exists
    experiment_details = mlflow_client.get_experiment_by_name(EXPERIMENT_NAME)

    if experiment_details is not None:
        experiment_id = experiment_details.experiment_id
    else:
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)

    # Start an MLFlow experiment run
    with mlflow.start_run(
        experiment_id=experiment_id, run_name="iris dataset rf run"
    ) as run:
        # Log parameters

        mlflow.log_param("max_depth", 10)
        mlflow.log_param("random_state", 0)
        mlflow.log_param("n_estimators", 100)
        clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
        clf.fit(X_train, y_train)
        iris_predict_y = clf.predict(X_test)

        roc_auc_score_val = roc_auc_score(
            y_test, clf.predict_proba(X_test), multi_class="ovr"
        )
        mlflow.log_metric("test roc_auc_score", roc_auc_score_val)

        # Log model
        result = mlflow.sklearn.log_model(clf, artifact_path="model")

        # Register a new version
    result = mlflow.register_model(result.model_uri, MODEL_NAME)

    client = MlflowClient()
    client.transition_model_version_stage(
        name=MODEL_NAME, version=result.version, stage=stage
    )


for i in range(5):
    main(MODEL_NAME=f"iris demo{i}")

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  
IRIS train df shape
(105, 4)
(105,)
IRIS test df shape
(45, 4)
(45,)


Registered model 'iris demo0' already exists. Creating a new version of this model...
2022/12/19 14:32:24 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris demo0, version 2
Created version '2' of model 'iris demo0'.


   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  
IRIS train df shape
(105, 4)
(105,)
IRIS test df shape
(45, 4)
(45,)


Registered model 'iris demo1' already exists. Creating a new version of this model...
2022/12/19 14:32:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris demo1, version 2
Created version '2' of model 'iris demo1'.


   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  
IRIS train df shape
(105, 4)
(105,)
IRIS test df shape
(45, 4)
(45,)


Registered model 'iris demo2' already exists. Creating a new version of this model...
2022/12/19 14:33:02 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris demo2, version 2
Created version '2' of model 'iris demo2'.


   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  
IRIS train df shape
(105, 4)
(105,)
IRIS test df shape
(45, 4)
(45,)


Registered model 'iris demo3' already exists. Creating a new version of this model...
2022/12/19 14:33:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris demo3, version 2
Created version '2' of model 'iris demo3'.


   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  
IRIS train df shape
(105, 4)
(105,)
IRIS test df shape
(45, 4)
(45,)


Registered model 'iris demo4' already exists. Creating a new version of this model...
2022/12/19 14:33:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris demo4, version 2
Created version '2' of model 'iris demo4'.


# write deployment file and commit to git repository

In [ ]:
! git clone https://github.com/rocket9-code/model-deployments

In [ ]:
dep_yaml = """apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: mlflow-var-test1
spec:
  name: iris
  predictors:
  - graph:
      children:
        - name: step-one
          modelUri: '{{ mlflow.blob["iris demo1"] }}'
          envSecretRefName: seldon-rclone-secret
          implementation: MLFLOW_SERVER
          type: MODEL
          children: 
              - name: step-two
                modelUri: '{{ mlflow.blob["iris demo2"] }}'
                envSecretRefName: seldon-rclone-secret
                implementation: MLFLOW_SERVER
                type: MODEL
                children: []
        - name: step-three
          implementation: MLFLOW_SERVER
          modelUri: '{{ mlflow.blob["iris demo3"] }}'
          envSecretRefName: seldon-rclone-secret
          type: MODEL
          children: []
      implementation: MLFLOW_SERVER
      modelUri: '{{ mlflow.blob["iris demo4"] }}'
      envSecretRefName: seldon-rclone-secret
      logger:
        url: http://broker-ingress.knative-eventing.svc.cluster.local/demo/default
        mode: all
      name: classifier
    name: default
    replicas: 1"""
with open("model-deployments/staging/seldon-deploy-test1.yaml", "x") as f:
    f.write(dep_yaml)

In [ ]:
! cd model-deployments &&  git add staging/seldon-deploy-test1.yaml

In [ ]:
! cd model-deployments &&  git commit -m "test deploy yaml" 
! cd model-deployments &&  git push

# wait for the controller to pickup the changes and creates a new deploy yaml

In [13]:
import time

from kubernetes import client as KubeClient
from kubernetes import config

try:
    config.load_kube_config()
except config.ConfigException:
    config.load_incluster_config()
kube_client = KubeClient.CustomObjectsApi()

you can see the controller updated the model uri with latest model versions

In [15]:
manifest = kube_client.get_namespaced_custom_object(
    group="machinelearning.seldon.io",
    version="v1",
    plural="seldondeployments",
    namespace="staging",
    name="mlflow-var",
)
demo1 = manifest["spec"]["predictors"][0]["graph"]["children"][0]["modelUri"]
demo2 = manifest["spec"]["predictors"][0]["graph"]["children"][0]["children"][0][
    "modelUri"
]
demo3 = manifest["spec"]["predictors"][0]["graph"]["children"][1]["modelUri"]
demo4 = manifest["spec"]["predictors"][0]["graph"]["modelUri"]

print(demo1, demo2, demo3, demo4)

wasbs://artifacts/mlflow/8/4083c71c946e47e19422218b69a5d67c/artifacts/model wasbs://artifacts/mlflow/9/10e8b48f3cfc451da361fabccb6e1c08/artifacts/model wasbs://artifacts/mlflow/10/262bee84b7dd4b039973084383880b57/artifacts/model wasbs://artifacts/mlflow/11/0dd0c915e3e0446d9139fb81b0b6ad83/artifacts/model


In [16]:
! helm delete mlflow-controller-staging -n mlflow

release "mlflow-controller-staging" uninstalled
